In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
import torch

OUTPUT_DIR = r"G:\My Drive\ev range prediction project agni\Project EV range prediction\Simulation\output"

# SUMO raw XML outputs
BATTERY_FILE = os.path.join(OUTPUT_DIR, "battery.xml")
FCD_FILE = os.path.join(OUTPUT_DIR, "fcd.xml")

# EV parameters (from ev_types.add.xml)
CAPACITY_WH = 42200  

# Output dataset paths
SAVE_PATH = os.path.join(OUTPUT_DIR, "ev_sumo_dataset.csv")
PICKLE_PATH = os.path.join(OUTPUT_DIR, "ev_sumo_dataset.pkl")

# Final 16-feature dataset
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "ev_sumo_dataset_16features.csv")

##### PARSE SUMO BATTERY OUTPUT

In [2]:
def parse_battery_data(file_path):
    """Parse SUMO battery.xml to extract charge, SOC, and power data."""
    tree = ET.parse(file_path)
    root = tree.getroot()

    records = []
    for timestep in root.findall("timestep"):
        time = float(timestep.get("time"))
        for veh in timestep.findall("vehicle"):
            records.append({
                "time": time,
                "vehicle_id": veh.get("id"),
                "chargeLevel": float(veh.get("chargeLevel", 0)),
                "power": float(veh.get("actualBatteryPower", 0)),
                "recuperation": float(veh.get("recuperationBatteryPower", 0))
            })

    df_batt = pd.DataFrame(records)
    df_batt["SOC"] = df_batt["chargeLevel"] / CAPACITY_WH
    return df_batt


##### PARSE SUMO FCD OUTPUT (SPEED + ACCEL)

In [3]:
def parse_fcd_data(file_path):
    """Parse SUMO fcd.xml to extract motion variables."""
    tree = ET.parse(file_path)
    root = tree.getroot()

    records = []
    for timestep in root.findall("timestep"):
        time = float(timestep.get("time"))
        for veh in timestep.findall("vehicle"):
            records.append({
                "time": time,
                "vehicle_id": veh.get("id"),
                "speed": float(veh.get("speed", 0)),
                "acceleration": float(veh.get("acceleration", 0))
            })

    return pd.DataFrame(records)

##### MERGE + CLEAN RAW DATA

In [4]:
def merge_datasets(df_batt, df_fcd):
    """Merge battery and FCD data into a unified dataset."""
    df = pd.merge(df_batt, df_fcd, on=["time", "vehicle_id"], how="inner")
    df.sort_values(by=["vehicle_id", "time"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Filter invalid SOC values
    df = df[(df["SOC"] >= 0) & (df["SOC"] <= 1)]

    # Convert and normalize
    df["speed_kmh"] = df["speed"] * 3.6
    df["power_kw"] = df["power"] / 1000.0

    print(f"✅ Final merged dataset: {len(df)} rows | {df['vehicle_id'].nunique()} vehicles")
    return df


##### DERIVE PHYSICS-INFORMED FEATURES (16)

In [ ]:
def derive_physics_features(df):
    """Generate the full 16 model input features."""
    # --- Physical constants ---
    MASS = 1320          # kg
    G = 9.81             # m/s²
    Crr = 0.01           # rolling resistance coefficient
    Cd = 0.29            # drag coefficient
    A = 2.38             # frontal area (m²)
    AIR_DENSITY = 1.225  # kg/m³
    TEMP = 25.0          # °C

    df.sort_values(by=["vehicle_id", "time"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # --- Basic physics ---
    df["velocity_ms"] = df["speed_kmh"] / 3.6
    df["rolling_resistance_force"] = MASS * G * Crr
    df["aero_drag_force"] = 0.5 * AIR_DENSITY * Cd * A * (df["velocity_ms"] ** 2)

    # --- Temporal / lag features ---
    df["power_kw_prev"] = df.groupby("vehicle_id")["power_kw"].shift(1).fillna(df["power_kw"])
    df["soc_prev"] = df.groupby("vehicle_id")["SOC"].shift(1).fillna(df["SOC"])

    # --- Instantaneous energy rate ---
    df["energy_rate"] = df["power_kw"] / df["speed_kmh"].replace(0, np.nan)
    
    df["energy_rate_prev"] = df.groupby("vehicle_id")["energy_rate"].shift(1).fillna(df["energy_rate"]).clip(0, 1)

    # --- Trip distance normalization ---
    df["delta_time"] = df.groupby("vehicle_id")["time"].diff().fillna(0.1)
    df["distance_m"] = df["velocity_ms"] * df["delta_time"]
    df["trip_distance_norm"] = df.groupby("vehicle_id")["distance_m"].cumsum()
    df["trip_distance_norm"] /= df["trip_distance_norm"].max()

    # --- Cumulative energy usage ---
    df["cumulative_energy_used"] = (
        df.groupby("vehicle_id")["power_kw"]
          .transform(lambda x: np.cumsum(np.maximum(x, 0)))
    )

    # --- Rolling averages ---
    window = 10
    df["velocity_mean_window"] = df.groupby("vehicle_id")["velocity_ms"].transform(lambda x: x.rolling(window, min_periods=1).mean())
    df["accel_mean_window"] = df.groupby("vehicle_id")["acceleration"].transform(lambda x: x.rolling(window, min_periods=1).mean())
    df["power_mean_window"] = df.groupby("vehicle_id")["power_kw"].transform(lambda x: x.rolling(window, min_periods=1).mean())

    # --- Static / contextual features ---
    df["elevation_grade"] = 0.0
    df["road_curvature"] = 0.0
    df["ambient_temperature"] = TEMP
    df["acceleration_ms2"] = df["acceleration"]
    df["time_of_day_norm"] = (df["time"] % 86400) / 86400.0

    # --- Reorder columns to match PINN input format ---
    ordered_cols = [
        "velocity_kmh", "elevation_grade", "road_curvature", "ambient_temperature",
        "acceleration_ms2", "rolling_resistance_force", "aero_drag_force",
        "power_kw_prev", "soc_prev", "energy_rate_prev", "trip_distance_norm",
        "velocity_mean_window", "accel_mean_window", "power_mean_window",
        "cumulative_energy_used", "time_of_day_norm"
    ]

    df["velocity_kmh"] = df["speed_kmh"]
    df_final = df[ordered_cols + ["SOC", "power_kw"]]
    df_final.to_csv(OUTPUT_PATH, index=False)

    print(f"\n✅ Saved 16-feature dataset:\n{OUTPUT_PATH}")
    print(df_final.head(10))
    return df_final


In [6]:
print("\n🔍 Parsing SUMO XML outputs...")
df_batt = parse_battery_data(BATTERY_FILE)
df_fcd = parse_fcd_data(FCD_FILE)


🔍 Parsing SUMO XML outputs...


In [7]:
print(f"Battery data: {len(df_batt)} rows | FCD data: {len(df_fcd)} rows")

df_merged = merge_datasets(df_batt, df_fcd)
df_merged.to_csv(SAVE_PATH, index=False)
df_merged.to_pickle(PICKLE_PATH)

Battery data: 470807 rows | FCD data: 470807 rows
✅ Final merged dataset: 470807 rows | 3587 vehicles


In [8]:
print(f"\n💾 Saved intermediate dataset:")
print(f" - CSV: {SAVE_PATH}")
print(f" - PKL: {PICKLE_PATH}")

print("\n⚙️ Deriving 16 physics-informed features...")
derive_physics_features(df_merged)

print("\n✅ All done! Your dataset is ready for EVRangePINN validation.")



💾 Saved intermediate dataset:
 - CSV: G:\My Drive\ev range prediction project agni\Project EV range prediction\Simulation\output\ev_sumo_dataset.csv
 - PKL: G:\My Drive\ev range prediction project agni\Project EV range prediction\Simulation\output\ev_sumo_dataset.pkl

⚙️ Deriving 16 physics-informed features...

✅ Saved 16-feature dataset:
G:\My Drive\ev range prediction project agni\Project EV range prediction\Simulation\output\ev_sumo_dataset_16features.csv
   velocity_kmh  elevation_grade  road_curvature  ambient_temperature  \
0         0.000              0.0             0.0                 25.0   
1         9.324              0.0             0.0                 25.0   
2        18.576              0.0             0.0                 25.0   
3        27.828              0.0             0.0                 25.0   
4        29.628              0.0             0.0                 25.0   
5        24.552              0.0             0.0                 25.0   
6        24.480         